# Ciencia de datos (CC408) - Primavera 2024

## Trabajo Práctico 3

### Parte I: Limpieza de la base

In [ ]:
!pip install openpyxl

In [345]:
# Importamos las librerías a usar

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## 0. LECTURA DEL DOCUMENTO
df = pd.read_excel('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 3')

#Dirección de Maggie: "C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 3"
#Dirección de Peter: "C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 3/usu_individual_T124.xlsx"

In [ ]:
# Cargar tu DataFrame (por ejemplo, desde un archivo CSV)
df = pd.read_excel('/content/drive/MyDrive/TP3 Ciencia de Datos/EPH_usu_1_Trim_2024_xls/usu_hogar_T124.xlsx')

# Especifica la columna y los valores que deseas eliminar
columna = 'nombre_columna'  # Reemplaza con el nombre real de tu columna
valores_a_eliminar = ['valor1', 'valor2']  # Reemplaza con los valores que deseas eliminar

# Filtrar el DataFrame
df_filtrado = df[~df[columna].isin(valores_a_eliminar)]

# Guardar el DataFrame filtrado si es necesario
df_filtrado.to_csv('archivo_filtrado.csv', index=False)

# Mostrar el DataFrame filtrado
print(df_filtrado)

### Parte II: Visualizaciones

In [ ]:
#COMPOSICIÓN POR SEXO PARA 2004 Y 2024

In [ ]:
#MATRIZ DE CORRELACIÓN

In [ ]:
#COUNT: DESOCUPADOS, INACTIVOS, OCUPADOS

In [ ]:
#INGRESO PER CÁPITA FAMILIAR SEGÚN ESTADO (OCUPADO, DESOCUPADO, INACTIVO)